In [1]:
from Bio import AlignIO, Phylo
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor

In [ ]:
TREEDIR="/data/shanlan/crc_trio/1.sub_analysis/runtime/2.sgWGS/tree/"
PHYDIR=OUTDIR=TREEDIR
samples=['FAP03P2_SNP_vd7_vr4','B139_SNP_vd7_vr4']

"""
NJ tree based on the distance of mutation distribution among different samples.
!!! This step was done by Phylo.TreeConstrunction.
construnct snv tree for each samples
"""
for mySample in samples:
    print(mySample)
    alignment = AlignIO.read(PHYDIR + mySample+".fa", "fasta") # Load the alignment from a Phylip file 
    calculator = DistanceCalculator('identity') # Calculate the distance matrix
    distance_matrix = calculator.get_distance(alignment)
    constructor = DistanceTreeConstructor() # Create a DistanceTreeConstructor object
    tree = constructor.nj(distance_matrix) # Build the tree using the neighbor-joining method
    # Root the tree at the specified outgroup
    try:
        tree.root_with_outgroup("ref")
        print("Tree has been successfully re-rooted using 'ref' as the outgroup.")
    except ValueError as e:
        print(f"Error rooting the tree with specified outgroup 'ref': {e}")
    # Optionally, save the rooted tree to a Newick file
    with open(OUTDIR+mySample+".nj.nwk", "w") as output:
        Phylo.write(tree, output, "newick")


In [ ]:
from ete3 import Tree
# from ete3 import TreeStyle
import os
import matplotlib as plt
from Bio import Phylo
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
"""
!!! This step was done after iqtree/phylo tree reconstruction.
rescale tree branch length in mutation numbers
"""
for mySample in samples:
    print(mySample)
    mut_nums = pd.read_csv(PHYDIR + mySample+".phy", delimiter = " ", skiprows=1,header = None, names = ["name","bi"], index_col=0)
    mut_nums["numMuts"] = [mut_nums["bi"][i].count("1") for i in range(mut_nums.shape[0])]
    tree = Phylo.read(TREEDIR+mySample+".nj"+".nwk", "newick") # from phylo
    tree_depths = tree.depths()
    depths = []
    mutations = []
    for i in tree_depths:
        if i.name in mut_nums.index:
            depths.append(tree_depths[i])
            mutations.append(mut_nums.loc[i.name].numMuts)
    depths = np.array(depths)
    mutations = np.array(mutations)
    print(mutations)

    reg = LinearRegression(fit_intercept=False).fit(depths.reshape(-1,1), mutations)
    scale = reg.coef_[0]
    #scale=59242 #unique mutations across all samples
    print(scale)

    for clade in tree.get_terminals():
        clade.branch_length = float(scale * clade.branch_length)
    for clade in tree.get_nonterminals()[1:]:
        clade.branch_length = float(scale * clade.branch_length)

    Phylo.write(tree, OUTDIR+mySample+".nj_rescale.nwk","newick") # from phylo ,nj
